In [ ]:
from dataclasses import dataclass

import os

import math

import polars as pl
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import kaggle_evaluation.default_inference_server

import torch
import torch.nn as nn # defining our neural network
import torch.optim as optim # training our neural network
from torch.utils.data import DataLoader, Dataset # loading data in batches

## Config

In [ ]:
@dataclass
class Config:

    # FFN learning rate
    learning_rate = 5e-4
    momentum = 0.95

    train_path = "/kaggle/input/hull-tactical-market-prediction/train.csv"
    test_path = "/kaggle/input/hull-tactical-market-prediction/test.csv"

    target_column = "market_forward_excess_returns"

    signal_multiplier: float = 16.0
    min_signal : float = 0.0 
    max_signal : float = 2.0
    

## Loading and Processing Data

In [ ]:

# Load data
train = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv")
test = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/test.csv")

# Columns to keep
test_cols = ["M1", "E1", "I1", "P1", "V1", "S1", "D1"]
train_cols = ["forward_returns", "risk_free_rate", "market_forward_excess_returns"]
base_col = ["date_id"]

# Apply filtering
train_filtered = train[base_col + test_cols + train_cols]
test_filtered = test[base_col + test_cols]

window = 50 # number of days in each normalization window

# rolling_min = train_filtered["market_forward_excess_returns"].rolling(window).min()
# rolling_max = train_filtered["market_forward_excess_returns"].rolling(window).max()

# train_filtered["target_market_norm"] = (
#     (train_filtered["forward_returns"] - rolling_min) / (rolling_max - rolling_min)
# )
# train_filtered["target_market_norm"].bfill(inplace=True)

# remove duplicate colums
train_filtered = train_filtered.loc[:, ~train_filtered.columns.duplicated()]
test_filtered = test_filtered.loc[:, ~test_filtered.columns.duplicated()]

print(f"Number of rows in train: {len(train_filtered)}")
print(f"Number of rows in test: {len(test_filtered)}")


In [ ]:
# Count number of rows with at least one NaN
num_nan_train = train_filtered.isna().any(axis=1).sum()
num_nan_test = test_filtered.isna().any(axis=1).sum()

print(f"Number of rows with NaN in train: {num_nan_train}")
print(f"Number of rows with NaN in test: {num_nan_test}")

X = train_filtered[test_cols]  # features
y = train_filtered[Config.target_column]  # target

mask = X.isna().sum(axis=1) == 0
X = X[mask]
y = y[mask]

In [ ]:
X.shape, y.shape

In [ ]:
X.head()

In [ ]:
y.head()

## FFN Model Definition

In [ ]:

# our neural network class is a subclass of nn.Module
# this handles a lot of the boilerplate code for us
# including parameter initialization, etc.
class FFN(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):

        '''
        Args:
            input_size: size of the input
            hidden_size: size of the hidden layer
            output_size: size of the output layer
        '''

        super().__init__() # call the parent class's constructor

        # define layers
        # nn.Linear takes in the size of the input and output
        self.fc1 = nn.Linear(input_size, hidden_size) # first fully connected layer
        self.relu = nn.ReLU() # activation function
        self.fc2 = nn.Linear(hidden_size, output_size) # second fully connected layer

    def forward(self, x):
        # define the forward pass
        out = self.fc1(x) # pass through first layer
        out = self.relu(out) # apply activation function
        out = self.fc2(out) # pass through second layer
        return out



## Dataset Definition

In [ ]:

class SP500Dataset(Dataset):
    def __init__(self, data, target_column):
        # data is a dataframe
        self.dataframe = data
        self.target = target_column

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # return a tuple of (features, target)
        return torch.tensor(self.dataframe.iloc[idx].values, dtype=torch.float32), \
            torch.tensor(self.target.iloc[idx], dtype=torch.float32)



## Training Loop

In [ ]:

df, labels = X, y

dataset = SP500Dataset(X, y)

# This is an iterable that will yield batches of data
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Our model, optimizer, and loss function
model = FFN(input_size=len(X.columns), hidden_size=5, output_size=1)

# Variant of Stochastic Gradient Descent
optimizer = optim.SGD(
    model.parameters(),
    lr=Config.learning_rate,
    momentum=Config.momentum
)

# Mean Squared Error Loss for regression tasks
criterion = nn.MSELoss()


loss_graph = []
for epoch in range(10): # loop over the dataset multiple times
    for inputs, targets in dataloader:
        optimizer.zero_grad() # zero the parameter gradients

        outputs = model(inputs) # forward pass
        loss = criterion(outputs.squeeze(), targets) # compute loss
        loss.backward() # backward pass
        optimizer.step() # update parameters
        loss_graph.append(loss.item())

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')



In [ ]:
plt.plot(loss_graph)
plt.show()

## Prediction

In [ ]:
def convert_ret_to_signal(
    ret_arr: np.ndarray,
) -> np.ndarray:
    """
    Converts raw model predictions (expected returns) into a trading signal.

    Args:
        ret_arr (np.ndarray): The array of predicted returns.
        params (RetToSignalParameters): Parameters for scaling and clipping the signal.

    Returns:
        np.ndarray: The resulting trading signal, clipped between min and max values.
    """
    return np.clip(
        ret_arr * Config.signal_multiplier + 1, Config.min_signal, Config.max_signal
    )

In [ ]:
def predict(test: pl.DataFrame) -> float:
    data = test.to_pandas()
    data = torch.tensor(data[test_cols].copy().to_numpy(),dtype=torch.float32)
    y_pred = model.forward(data)
    pred = float(y_pred.item())
    print(pred)
    signal = convert_ret_to_signal(pred)
    print(signal)
    return signal
    


In [ ]:
# When your notebook is run on the hidden test set, inference_server.serve must be called within 15 minutes of the notebook starting
# or the gateway will throw an error. If you need more than 15 minutes to load your model you can do so during the very
# first `predict` call, which does not have the usual 1 minute response deadline.
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))